# Sea Surface Temperature from Remote Zarr Store

This example demonstrates using `PyVistaXarraySource` to lazily visualize a large, cloud-hosted sea surface temperature (SST) dataset stored as Zarr. The data is accessed on-demand — only the portion needed for rendering is loaded into memory.

The `resolution` parameter controls how many points are sampled from the full dataset, enabling fast previews of otherwise massive data.

In [ ]:
import xarray as xr

from pvxarray.vtk_source import PyVistaXarraySource

## Load Remote Data

Open the NOAA CoastWatch Geo-Polar SST dataset from the Pangeo cloud data catalog. Using `chunks={}` enables dask-backed lazy loading:

In [ ]:
url = "https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/noaa-coastwatch-geopolar-sst-feedstock/noaa-coastwatch-geopolar-sst.zarr"
dataset = xr.open_dataset(url, engine="zarr", chunks={})

# Configure the VTK source for lazy evaluation
algorithm = PyVistaXarraySource(
    data_array=dataset["analysed_sst"],
    x="lat",
    y="lon",
    time="time",
    resolution=0.01,  # Sample 1% of data for fast preview
)
algorithm.time_index = 0

## Render the SST Field

Call `.apply()` to evaluate the source and produce a PyVista mesh. Because `resolution=0.01`, only ~1% of the data is loaded:

In [ ]:
mesh = algorithm.apply()
mesh.plot()